# Piping questions and answers
This notebook demonstrates how to pipe components of a question and answer into follow-on questions.
Survey questions are automatically administered in the required order to facilitate the piping.

We also demonstrate how to modify question instructions.

Learn more about each of these topics in the [docs](https://docs.expectedparrot.com/en/latest/):
* Constructing [questions](https://docs.expectedparrot.com/en/latest/questions.html) with [optional parameters](https://docs.expectedparrot.com/en/latest/questions.html#optional-parameters)
* Adding logic, rules and memory to [surveys](https://docs.expectedparrot.com/en/latest/surveys.html)
* Designing AI [agents](https://docs.expectedparrot.com/en/latest/agents.html)
* Examining question (user) and agent (system) [prompts](https://docs.expectedparrot.com/en/latest/prompts.html)
* Selecting [language models](https://docs.expectedparrot.com/en/latest/language_models.html) to generate results

Before running the code below, please see instructions on [installing](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library and storing [API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models.

In [1]:
from edsl import QuestionMultipleChoice, QuestionList, QuestionFreeText, Survey, Model

In [2]:
m = Model("gemini-1.5-flash")

Here we use the optional parameter `answering_instructions` to provide a persona for a single question.
Because this parameter replaces the default instructions for the model to also providing a `comment` on its answer (*"After the answer, you can put a comment explaining why you chose that option on the next line."*), we include those instructions as well so that the `comment` field will still be populated in the results:

In [3]:
q1 = QuestionMultipleChoice(
    question_name = "season",
    question_text = "What is your favorite season?",
    question_options = ["Spring", "Summer", "Fall", "Winter"],
    answering_instructions = """
    After the answer, you can put a comment explaining why you chose 
    that option on the next line. 
    Answer the question as an alpine skier.
    """
)

In [4]:
q2 = QuestionList(
    question_name = "followup",
    question_text = """
    Review the following interview and then propose 3 followup 
    questions for the interviewer to ask:
    
    Interviewer: '{{ season.question_text }}'
    Interviewee: '{{ season.answer }}. {{ season.comment }}'
    """,
    max_list_items = 3
)

There is no "comment" field for free text questions, so we do not include any instructions for it in the `answering_instruction`:

In [5]:
q3 = QuestionFreeText(
    question_name = "followup0",
    question_text = "{{ followup.answer[0] }}",
    answering_instructions = "Answer the question as an alpine skier. Max characters: 100"
)
q4 = QuestionFreeText(
    question_name = "followup1",
    question_text = "{{ followup.answer[1] }}",
    answering_instructions = "Answer the question as an alpine skier. Max characters: 100"
)
q5 = QuestionFreeText(
    question_name = "followup2",
    question_text = "{{ followup.answer[2] }}",
    answering_instructions = "Answer the question as an alpine skier. Max characters: 100"
)

In [6]:
survey = Survey([q1, q2, q3, q4, q5])

We can add a "memory" of the first question and answer and each prior followup question and answer when we administer each successive followup question (skipping the question where we generated the followup question texts):

In [7]:
job = (
    survey
    .add_targeted_memory(q3, q1)
    .add_memory_collection(q4, [q1, q3])
    .add_memory_collection(q5, [q1, q3, q4])
    .by(m)
)

We can examine the question (user) prompts before running the survey, noting that piped inputs are not yet available. There is no system prompt because we have not created any agents:

In [8]:
job.prompts().select("user_prompt")

,user_prompt
0,"What is your favorite season? Spring Summer Fall Winter Only 1 option may be selected. After the answer, you can put a comment explaining why you chose that option on the next line. Answer the question as an alpine skier."
1,"Review the following interview and then propose 3 followup questions for the interviewer to ask: Interviewer: 'What is your favorite season?' Interviewee: '<>. <>' The list must not contain more than 3 items. Return your answers on one line, in a comma-separated list of your responses, with square brackets and each answer in quotes E.g., [""A"", ""B"", ""C""] After the answers, you can put a comment explaining your choice on the next line."
2,"Answer the question as an alpine skier. Max characters: 100 Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None"
3,"Answer the question as an alpine skier. Max characters: 100 Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None Prior questions and answers: Question: Answer: None"
4,"Answer the question as an alpine skier. Max characters: 100 Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None Prior questions and answers: Question: Answer: None Prior questions and answers: Question: Answer: None"


In [9]:
results = job.run()

Job UUID,51923e26-84b2-4371-904d-7a5ec747b8ac
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/51923e26-84b2-4371-904d-7a5ec747b8ac
Error Report URL,None
Results UUID,52b0c172-839f-4d80-8446-26dd4076ddd4
Results URL,https://www.expectedparrot.com/content/52b0c172-839f-4d80-8446-26dd4076ddd4


In [10]:
results.select("season", "season_comment", "followup", "followup_comment", "followup0", "followup1", "followup2")

,answer.season,comment.season_comment,answer.followup,comment.followup_comment,answer.followup0,answer.followup1,answer.followup2
0,Winter,Because that's when the snow is best for skiing!,"['Can you elaborate on why you enjoy winter the most?', 'What activities do you typically engage in during the winter season?', 'How does your preference for winter influence your work style or approach to projects?']","These follow-up questions aim to delve deeper into the interviewee's personality, interests, and work ethic, all while using the seemingly innocuous initial question as a springboard. They move beyond a simple preference to explore underlying motivations and potential connections to the job.","The crisp air, the untouched powder, the exhilarating speed carving down a mountain – winter's my playground! It's pure freedom.","Mostly skiing, of course! I also enjoy après-ski, tuning my skis, and planning my next trip to the mountains.","Winter's precision and power inform my work. I approach projects with focused intensity, like carving a perfect turn. I strategize meticulously, anticipating challenges as I would icy patches, then execute with swift efficiency."


## Posting to Coop

In [11]:
from edsl import Notebook

n = Notebook("piping_comments.ipynb")
info = n.push(description = "Piping example")
info

{'description': 'Piping example',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/25d0e86a-1f12-4b84-bbc4-c5595cb96f6d',
 'uuid': '25d0e86a-1f12-4b84-bbc4-c5595cb96f6d',
 'version': '0.1.39.dev2',
 'visibility': 'unlisted'}

Updating the notebook at Coop

In [13]:
n = Notebook("piping_comments.ipynb") # resave
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}